In [1]:
import pandas as pd
import numpy as np

In [3]:
%%timeit
#Reading the file
xlsx = pd.ExcelFile("gorilla_test_data.xlsx")
meter_df = xlsx.parse(sheet_name=0)
forecast_df = xlsx.parse(sheet_name=1,parse_dates=True)
rate_df = xlsx.parse(sheet_name=2,parse_dates=True)
#Final dataframe structure
total_df = pd.DataFrame({'meter_id': meter_df['meter_id'],
                         'Total Estimated Consumption (kwh)' : pd.Series(dtype='float'),
                         'Total Cost (£)': pd.Series(dtype='float')
                         })
# Filtering the rate table
rate_df.dropna(axis=0,inplace=True)
rate_df.dropna(axis=0,inplace=True)
aq_dict = dict(zip(meter_df['exit_zone'],meter_df['aq_kwh']))
rate_df['aq_kwh'] = rate_df['exit_zone'].map(aq_dict)
updated_rate = rate_df[(rate_df['aq_min_kwh'] <= rate_df['aq_kwh']) &
                     (rate_df['aq_max_kwh'] > rate_df['aq_kwh'])
                    ]
final_rate = updated_rate.drop(['aq_min_kwh','aq_max_kwh','aq_kwh'],axis=1)
ez_dict = dict(zip(meter_df['meter_id'],meter_df['exit_zone']))
# Reshaping the forecast and rate table
f_pivot = forecast_df.pivot(index='date',columns='meter_id',values='kwh')
r_pivot = final_rate.pivot(index='date',columns='exit_zone',values='rate_p_per_kwh')
r_sampled = r_pivot.asfreq(freq='d').ffill()
#Finding the final forecast table
m_df = f_pivot.merge(r_sampled,how='inner',on='date')
#Finding Total dataframe
for id,ez in ez_dict.items():
  aq = m_df[id].values
  rate = m_df[ez].values
  total_df['Total Estimated Consumption (kwh)'] = np.where(total_df['meter_id'] == id ,(aq.sum()).round(2),total_df['Total Estimated Consumption (kwh)'])
  total_df['Total Cost (£)'] = np.where(total_df['meter_id'] == id ,((np.dot(aq,rate))*.01).round(2),total_df['Total Cost (£)'])

print(total_df)

   meter_id  Total Estimated Consumption (kwh)  Total Cost (£)
0  14676236                            28978.0          100.15
1  34509937                            78324.0          275.49
2  50264822                           265667.0          731.24
3  88357331                           484399.0         1433.16
   meter_id  Total Estimated Consumption (kwh)  Total Cost (£)
0  14676236                            28978.0          100.15
1  34509937                            78324.0          275.49
2  50264822                           265667.0          731.24
3  88357331                           484399.0         1433.16
   meter_id  Total Estimated Consumption (kwh)  Total Cost (£)
0  14676236                            28978.0          100.15
1  34509937                            78324.0          275.49
2  50264822                           265667.0          731.24
3  88357331                           484399.0         1433.16
   meter_id  Total Estimated Consumption (kwh)  Total C